# 0.1 Settings and Importing Modules

In [1]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import pprint
import itertools
import pickle

from mlxtend.plotting import plot_decision_regions
from mlxtend.classifier import StackingClassifier # <-- note: this is not from sklearn!

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, accuracy_score, roc_auc_score, precision_recall_curve,f1_score, fbeta_score, confusion_matrix, roc_curve, log_loss, make_scorer
from sklearn.preprocessing import StandardScaler,PolynomialFeatures

from numpy import mean
from sklearn import svm, model_selection
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, GridSearchCV, RepeatedStratifiedKFold, cross_validate

from sklearn.naive_bayes import BernoulliNB, MultinomialNB, GaussianNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier, AdaBoostClassifier, BaggingRegressor
from sklearn.neighbors import KNeighborsClassifier

plt.style.use('ggplot')

%matplotlib inline

# 0.2 Loading Data

In [2]:
pd.set_option('display.max_rows', 150)
pd.set_option('display.float_format', '{:.2f}'.format)
np.set_printoptions(suppress=True)

In [3]:
# loading data from AWS
cnx = create_engine('postgresql://ubuntu@3.87.186.58:5432/steam3tag')

In [4]:
# checking if the dataset is loaded correctly
pd.read_sql_query('''SELECT * FROM steam_3t_strat LIMIT 5''', cnx)

OperationalError: (psycopg2.OperationalError) could not connect to server: Connection timed out (0x0000274C/10060)
	Is the server running on host "3.87.186.58" and accepting
	TCP/IP connections on port 5432?

(Background on this error at: http://sqlalche.me/e/e3q8)

In [ ]:
# some basic feature engineerings have been done and saved as views on AWS using DBeaver, here we will just load it
# for details about what kind of basic feature engineering is done, visit my Medium post: https://medium.com/@opophehu/supervised-classification-a0043d0c5ba5
basic_fe = pd.read_sql_query('''SELECT * FROM basic_fe''', cnx)
basic_fe

# 1.1 Cleaning Data
### Dropping NaN and setting floor threshold

##### 1.1.1: dropping unneeded entries

In [ ]:
# copy the feature engineered view for exploratory purposes
df = basic_fe.copy()
# dropping unneeded columns
df.drop(['platform', 'releasedate', 'alltags', 'discountpercentage'], axis=1, inplace = True)
# cleaning NaN values
df.fillna(0, inplace=True)
df.describe()

##### 1.1.2: Setting floor threshold at 25%

In [ ]:
# setting a floor threshold for review_count based on the information above at 25%
df=df[df.reviewcount>4]

##### 1.1.3: Splitting data

In [ ]:
# splitting the data and getting ready to explore
X, y = df.drop('onsale',axis=1), df['onsale']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

##### 1.1.4: Preliminary data viz

In [ ]:
# seeing how this data looks in terms of distinguishablity, in this case, it doesn't look good, extra engineering might need to be done
sns.pairplot(df, hue='onsale');

##### 1.1.5: Class imbalance

In [ ]:
# checking class imbalance
df['onsale'].value_counts()

In [ ]:
# checking class imbalance in terms of %
530/8135

In [ ]:
# getting class balance coefficients in order to properly train the model, in this case it would be 1:15 (not_on_sale : on_sale)
8135/530

# 2.1 Define Functions
### for classification, feature engineering, and feature selection

##### 2.1.1: Function to run classification models

In [ ]:
def classification(X, y, classweight={0:1, 1:1}):
    ''' 
    A function to run all classification models.
    It also automatically filter and selects the best model based on predefined metric (precision, recall, f1). 
    It also prints out detailed information (scores and graph) from each model, including feature importances from Random Forest and Gradient Boosting. 

    Inputs:
    X = Your dataset without the target (y)
    y = Your target, whatever you are trying to predict ---Binary.
    classweight = Your class weights, default is 1 to 1.

    Output:
    Returns the best model based on predefined metric (precision, recall, f1).
    '''
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=4444)
    
    model_list=[]
    
    # KNN
    knn = KNeighborsClassifier(n_neighbors=5)
    knn.fit(X_train, y_train)
    
    model_list.append(knn)
    
    # Logistic
    lm = LogisticRegression(C=0.95, class_weight = classweight)
    lm.fit(X_train, y_train)
    lm_pred = lm.predict(X_test)
    
    model_list.append(lm)
    
    # Gaussian Naive Bayes
    gnb = GaussianNB()
    gnb.fit(X_train, y_train)
    
    model_list.append(gnb)
    
    # Support Vector Machine   
    svm = SVC(gamma='auto',probability=True, class_weight = classweight)
    svm.fit(X_train, y_train)
    
    model_list.append(svm)
    
    # Decision Tree
    tree = DecisionTreeClassifier(random_state=5, class_weight = classweight)
    tree.fit(X_train, y_train)
    
    model_list.append(tree)
    
    # Random Forest
    ran = RandomForestClassifier(random_state=5, class_weight = classweight)
    ran.fit(X_train, y_train)
    
    model_list.append(ran)
    
    # Gradient Boosting
    gb = GradientBoostingClassifier(n_estimators = 90, max_depth = 100)
    gb.fit(X_train, y_train)
    
    model_list.append(gb)
    
    best_f1_score = 0
    best_model = ran
    
    for model in model_list:
        print (type(model).__name__)
        print ('Accuracy: {:6.4f}'.format(accuracy_score(y_test, model.predict(X_test))))
        print("Precision: {:6.4f},   Recall: {:6.4f},   f1: {:6.4f}".format(precision_score(y_test, model.predict(X_test)), recall_score(y_test, model.predict(X_test)), f1_score(y_test, model.predict(X_test))), '\n')
        
        if f1_score(y_test, model.predict(X_test)) > best_f1_score:
            best_f1_score = f1_score(y_test, model.predict(X_test))
            best_model = model
        
        
        fpr, tpr, thresholds = roc_curve(y_test, model.predict_proba(X_test)[:,1])
        plt.plot(fpr, tpr,lw=2)
        plt.plot([0,1],[0,1],c='violet',ls='--')
        plt.xlim([-0.05,1.05])
        plt.ylim([-0.05,1.05])
        plt.xlabel('False positive rate')
        plt.ylabel('True positive rate')
        plt.title('ROC curve for ' + type(model).__name__);
        print("ROC AUC score = {:6.4f}".format(roc_auc_score(y_test, model.predict_proba(X_test)[:,1])))
        plt.show()
    
    k = list(X.columns)
#     k.remove(y.columns[0])
    pp = pprint.PrettyPrinter(indent=4)
    
    print('\n')
    print("Random Forest feature importances:",'\n')
    pp.pprint(sorted(list(zip(k, ran.feature_importances_)), key=lambda x: x[1], reverse=True))
    
    print('\n')
    print("Gradient Boosting feature importances:",'\n')
    pp.pprint(sorted(list(zip(k, gb.feature_importances_)), key=lambda x: x[1], reverse=True))
    
    print('\n')
    print("Best model is ", type(best_model).__name__)
    print("With f1 score of {:6.4f} and ROC AUC of {:6.4f}".format(f1_score(y_test, best_model.predict(X_test)), roc_auc_score(y_test, best_model.predict_proba(X_test)[:,1])))
    
    return best_model

##### 2.1.2: Function to do exploratory feature engineering

In [ ]:
def explore_fe(df, target):
    ''' 
    A function to do exploratory feature engineering. 
    It's flexible in its purpose, and is currently configured for this project only.

    Inputs:
    df (like X) = Your dataset without the target (y)
    target (like y) = Your target, whatever you are trying to predict ---Binary.

    Output:
    Returns engineered X (dataframe without target) based on the engineering logic.
    '''
    df = df.astype(float)
    df = df.replace({0:1 , 1:2})
    
    for i in range (0, len(df.columns)):
        df[f'{df.columns[i]}^2'] = np.square(df[df.columns[i]])
        df[f'{df.columns[i]}^1/2'] = np.sqrt(df[df.columns[i]])
        df[f'{df.columns[i]} * {df.columns[i+1]}'] = df[df.columns[i]] * df[df.columns[i+1]]
        df[f'{df.columns[i]} / {df.columns[i+1]}'] = df[df.columns[i]] / df[df.columns[i+1]]
#         df[f'{df.columns[i]} + {df.columns[i+1]}'] = df[df.columns[i]] + df[df.columns[i+1]]
#         df[f'{df.columns[i]} - {df.columns[i+1]}'] = df[df.columns[i]] - df[df.columns[i+1]]
        

#     df.fillna(0, inplace = True)
#     df.replace([np.inf, -np.inf], np.nan).dropna(axis=1)
    df[~df.isin([np.nan, np.inf, -np.inf]).any(1)].astype(np.float64)
    
    X,y= df, target
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=4444)
    
    ran = RandomForestClassifier(random_state=5)
    ran.fit(X_train, y_train)
    
    print ('Accuracy: ', accuracy_score(y_test, ran.predict(X_test)))
    print("Precision: {:6.4f},   Recall: {:6.4f},   f1: {:6.4f}".format(precision_score(y_test, ran.predict(X_test)), 
                                                 recall_score(y_test, ran.predict(X_test)), f1_score(y_test, ran.predict(X_test))), '\n')
    
    k = list(X.columns)
    pp = pprint.PrettyPrinter(indent=4)
    pp.pprint(sorted(list(zip(k, ran.feature_importances_)), key=lambda x: x[1], reverse=True))
    
    return df

##### 2.1.3: Function to poly transform columns

In [ ]:
def PolynomialFeatures_labeled(input_df,power):
    '''
    Basically this is a cover for the sklearn preprocessing function. 
    The problem with that function is if you give it a labeled dataframe, it ouputs an unlabeled dataframe with potentially
    a whole bunch of unlabeled columns. 

    Inputs:
    input_df = Your labeled pandas dataframe (list of x's not raised to any power) 
    power = what order polynomial you want variables up to. (use the same power as you want entered into pp.PolynomialFeatures(power) directly)

    Output:
    Output: This function relies on the powers_ matrix which is one of the preprocessing function's outputs to create logical labels and 
    outputs a labeled pandas dataframe   
    '''
    poly = PolynomialFeatures(power)
    output_nparray = poly.fit_transform(input_df)
    powers_nparray = poly.powers_

    input_feature_names = list(input_df.columns)
    target_feature_names = ["Constant Term"]
    for feature_distillation in powers_nparray[1:]:
        intermediary_label = ""
        final_label = ""
        for i in range(len(input_feature_names)):
            if feature_distillation[i] == 0:
                continue
            else:
                variable = input_feature_names[i]
                power = feature_distillation[i]
                intermediary_label = "%s^%d" % (variable,power)
                if final_label == "":         #If the final label isn't yet specified
                    final_label = intermediary_label
                else:
                    final_label = final_label + " x " + intermediary_label
        target_feature_names.append(final_label)
    output_df = pd.DataFrame(output_nparray, columns = target_feature_names)
    return output_df

##### 2.1.4: Function to select features in order to reduce noise

In [ ]:
from sklearn.feature_selection import SelectFromModel, RFE, SelectKBest, chi2
from sklearn.preprocessing import MinMaxScaler
from lightgbm import LGBMClassifier

In [ ]:
def feature_selection(X, y, score_to_keep = 5):
    '''
    A function to select features by votes of 6 models who can calculate feature importances.
    Also prints out how many original features there are, how many selected, and a list of selected features.
    Original idea from https://www.kaggle.com/mlwhiz/feature-selection-using-football-data

    Inputs:
    X = Your dataset without the target (y)
    y = Your target, whatever you are trying to predict --- Binary.
    score_to_keep = Pick features that have a 'score_to_keep' amount of votes --- max is 6 votes, default is 5. 
    
    Output:
    Returns selected_X as a dataframe without target(y). 
    '''
    feature_name = list(X.columns)
    num_feats=len(X.columns)
    def cor_selector(X, y,num_feats):
        cor_list = []
        feature_name = X.columns.tolist()
        # calculate the correlation with y for each feature
        for i in X.columns.tolist():
            cor = np.corrcoef(X[i], y)[0, 1]
            cor_list.append(cor)
        # replace NaN with 0
        cor_list = [0 if np.isnan(i) else i for i in cor_list]
        # feature name
        cor_feature = X.iloc[:,np.argsort(np.abs(cor_list))[-num_feats:]].columns.tolist()
        # feature selection? 0 for not select, 1 for select
        cor_support = [True if i in cor_feature else False for i in feature_name]
        return cor_support, cor_feature
    cor_support, cor_feature = cor_selector(X, y,num_feats)
    
    X_norm = MinMaxScaler().fit_transform(X)
    chi_selector = SelectKBest(chi2, k=num_feats)
    chi_selector.fit(X_norm, y)
    chi_support = chi_selector.get_support()
    chi_feature = X.loc[:,chi_support].columns.tolist()
    
    rfe_selector = RFE(estimator=LogisticRegression(), n_features_to_select=num_feats, step=10, verbose=5)
    rfe_selector.fit(X_norm, y)
    rfe_support = rfe_selector.get_support()
    rfe_feature = X.loc[:,rfe_support].columns.tolist()
    
    embeded_lr_selector = SelectFromModel(LogisticRegression(penalty="l2"), max_features=num_feats)
    embeded_lr_selector.fit(X_norm, y)
    embeded_lr_support = embeded_lr_selector.get_support()
    embeded_lr_feature = X.loc[:,embeded_lr_support].columns.tolist()
    
    embeded_rf_selector = SelectFromModel(RandomForestClassifier(n_estimators=100), max_features=num_feats)
    embeded_rf_selector.fit(X_norm, y)
    embeded_rf_support = embeded_rf_selector.get_support()
    embeded_rf_feature = X.loc[:,embeded_rf_support].columns.tolist()
    
    lgbc=LGBMClassifier(n_estimators=500, learning_rate=0.05, num_leaves=32, colsample_bytree=0.2,
            reg_alpha=3, reg_lambda=1, min_split_gain=0.01, min_child_weight=40)
    embeded_lgb_selector = SelectFromModel(lgbc, max_features=num_feats)
    embeded_lgb_selector.fit(X_norm, y)
    embeded_lgb_support = embeded_lgb_selector.get_support()
    embeded_lgb_feature = X.loc[:,embeded_lgb_support].columns.tolist()
    
    feature_selection_df = pd.DataFrame({'Feature':feature_name, 'Pearson':cor_support, 'Chi-2':chi_support, 'RFE':rfe_support, 'Logistics':embeded_lr_support,
                                        'Random Forest':embeded_rf_support, 'LightGBM':embeded_lgb_support})
    
    feature_selection_df['Total'] = np.sum(feature_selection_df, axis=1)
    
    feature_selection_df = feature_selection_df.sort_values(['Total','Feature'] , ascending=False)
    feature_selection_df.index = range(1, len(feature_selection_df)+1)
    
    
    selected_X = X.copy()
    to_drop = []
    for i in range (0, len(feature_selection_df)):
        if feature_selection_df.Total.values[i] < score_to_keep:
            to_drop.append(feature_selection_df.Feature.values[i])

    selected_X = selected_X.drop(to_drop, axis = 1)
    
    print ("Number of orginal features: ", num_feats)
    print ("Number of selected features: ", len(selected_X.columns), '\n')
    pp = pprint.PrettyPrinter(indent=4)
    print("Selected Features:")
    pp.pprint(list(selected_X.columns))
    
    
    return selected_X

# 3.1 Exploratory Model Building
### Logistic regression vs. Naive Bayes and exploratory grid search

##### 3.1.1: Grid search with logistic regression

In [ ]:
# define model
model = LogisticRegression(solver='lbfgs')

# define grid
balance = [{0:100,1:1}, {0:10,1:1}, {0:1,1:1}, {0:1,1:10}, {0:1,1:100}]
param_grid = dict(class_weight=balance)

# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

# define grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='roc_auc')

# execute the grid search
grid_result = grid.fit(X_train, y_train)

# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

##### 3.1.2: Explore skillfulness with this class weight

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, grid_result.predict_proba(X_test)[:,1])

plt.plot(fpr, tpr,lw=2)
plt.plot([0,1],[0,1],c='violet',ls='--')
plt.xlim([-0.05,1.05])
plt.ylim([-0.05,1.05])


plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve for games on sale');
print("ROC AUC score = ", roc_auc_score(y_test, grid_result.predict_proba(X_test)[:,1]))

##### 3.1.3: Compare against dummy classifier

In [ ]:
from sklearn.dummy import DummyClassifier

dc = DummyClassifier()
dc.fit(X_train, y_train)
print("Log-loss on logit: {:6.4f}".format(log_loss(y_test, grid_result.predict_proba(X_test))))
print("Log-loss on dummy classifier: {:6.4f}".format(log_loss(y_test, dc.predict_proba(X_test))))

In [ ]:
print("Logistic accuracy: {:6.4f}".format(grid_result.score(X_test, y_test)))
print("Dummy accuracy: {:6.4f}".format(dc.score(X_test, y_test)))

##### 3.1.4: Explore pure probability classification with Naive Bayes

In [ ]:
nb = GaussianNB()
nb.fit(X_train, y_train)
nb.score(X_test, y_test)

##### 3.1.5: Explore skillfulness with no class weight

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, nb.predict_proba(X_test)[:,1])

plt.plot(fpr, tpr,lw=2)
plt.plot([0,1],[0,1],c='violet',ls='--')
plt.xlim([-0.05,1.05])
plt.ylim([-0.05,1.05])


plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve for games on sale');
print("ROC AUC score = ", roc_auc_score(y_test, nb.predict_proba(X_test)[:,1]))

# 3.2 Establish baseline model

In [ ]:
baseline_model = classification(X, y, {0: 1, 1: 15})

# 4.1 Exploratory Feature Engineering
### Feature multiplication and Feature division

##### 4.1.1: Transform the dataset using custom algorithm

In [ ]:
explore_X = explore_fe(X, y)

##### 4.1.2: Compare metrics and determine effectiveness of the above F.E.

In [ ]:
classification(explore_X, y, {0: 1, 1: 15})

# 4.2 Feature Selection based on custom F.E.

##### 4.2.0: Test feature selection algorithm on pre engineered dataframe

In [ ]:
feature_selection(X, y)

##### 4.2.1: Filter Exploratory F.E.'d dataframe from 4.1 (features with at least 5 votes)

In [ ]:
exp_X_sel = feature_selection(explore_X, y)

##### 4.2.2: Run modeling algorithm to check metrics

In [ ]:
classification(exp_X_sel, y, {0: 1, 1: 15})

##### 4.2.3: Filter Exploratory F.E.'d dataframe from 4.1 (features with at least 6 (max) votes)

In [ ]:
exp_X_sel_6 = feature_selection(explore_X, y, 6)

##### 4.2.4: Run modeling algorithm to check metrics

In [ ]:
classification(exp_X_sel_6, y, {0: 1, 1: 15})

# 5.1 Poly FE
### Polynomial Transformation and feature selection

##### 5.1.1: Poly transform the original dataframe

In [ ]:
explore_X_poly=PolynomialFeatures_labeled(X,2)

##### 5.1.2: Check scores before feature selection

In [ ]:
classification(explore_X_poly, y, {0: 1, 1: 15})

##### 5.1.3: Select feature by either 5 or 6 votes

In [ ]:
exp_Xpoly_sel_5 = feature_selection(explore_X_poly, y, 5)
exp_Xpoly_sel_6 = feature_selection(explore_X_poly, y, 6)

##### 5.1.4: Run modeling algorithm to check metrics

In [ ]:
best_model=classification(exp_Xpoly_sel_5, y, {0: 1, 1: 15})

In [ ]:
classification(exp_Xpoly_sel_6, y, {0: 1, 1: 15})

# 6.1 Ensembling
### Explore the effectiveness of ensembling technique on this problem

##### 6.1.0: Setting up all the models that will be used in Soft Voting Classifier

In [ ]:
classweight = {0: 1, 1: 15}

lr_model = LogisticRegression(solver="lbfgs", class_weight= classweight)
nb_model = GaussianNB()
knn_model = KNeighborsClassifier()
svc_model = SVC(probability=True, gamma="scale", class_weight=classweight)
rf_model = RandomForestClassifier(n_estimators=100,class_weight=classweight)
et_model = ExtraTreesClassifier(n_estimators=100, class_weight=classweight)
ada_model = AdaBoostClassifier()

models = ["lr_model", "nb_model", "knn_model", "svc_model", "rf_model", "et_model", "ada_model"]

In [ ]:
for model_name in models:
    
    curr_model = eval(model_name)
    
    curr_model.fit(X_train, y_train)
    
    with open(f"models/{model_name}.pickle", "wb") as pfile:
        pickle.dump(curr_model, pfile)

In [ ]:
# Load pre-trained/tuned models

model_names = ["lr_model", "nb_model", "knn_model", "svc_model", "rf_model", "et_model", "ada_model"]

import os
# filename = "/tmp/not_exist/filenames.pkl"
# os.makedirs(os.path.dirname(filename), exist_ok=True)

for model_name in model_names:
    filename = f"models/{model_name}.pickle"
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    with open(filename, "rb") as pfile:
        exec(f"{model_name} = pickle.load(pfile)")

model_vars = [eval(n) for n in model_names]
model_list = list(zip(model_names, model_vars))

In [ ]:
# Quick peek at each model performance

for model_name in model_names:
    curr_model = eval(model_name)
    print(f'{model_name} score: {curr_model.score(X_test, y_test)}')

#### 6.1.1 Soft Voting Classifier

In [ ]:
model_list
# create voting classifier
voting_classifer = VotingClassifier(estimators=model_list,
                                    voting='soft', 
                                    n_jobs=-1)
voting_classifer.fit(X_train, y_train)
y_pred = voting_classifer.predict(X_test)
print ('Accuracy: ', accuracy_score(y_test, y_pred))
print("Precision: {:6.4f},   Recall: {:6.4f},   f1: {:6.4f}".format(precision_score(y_test, y_pred), recall_score(y_test, y_pred), f1_score(y_test, y_pred)), '\n')

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, voting_classifer.predict_proba(X_test)[:,1])
plt.plot(fpr, tpr,lw=2)
plt.plot([0,1],[0,1],c='violet',ls='--')
plt.xlim([-0.05,1.05])
plt.ylim([-0.05,1.05])
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve for ' + type(voting_classifer).__name__);
print("ROC AUC score = {:6.4f}".format(roc_auc_score(y_test, voting_classifer.predict_proba(X_test)[:,1])))
plt.show()

#### 6.1.2 Stacking Classifier

In [ ]:
stacked = StackingClassifier(
    classifiers=model_vars, meta_classifier=BernoulliNB(), use_probas=False)

In [ ]:
stacked.fit(X_train, y_train)

In [ ]:
y_pred = stacked.predict(X_test)
print ('Accuracy: ', accuracy_score(y_test, y_pred))
print("Precision: {:6.4f},   Recall: {:6.4f},   f1: {:6.4f}".format(precision_score(y_test, y_pred), recall_score(y_test, y_pred), f1_score(y_test, y_pred)), '\n')

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, stacked.predict_proba(X_test)[:,1])
plt.plot(fpr, tpr,lw=2)
plt.plot([0,1],[0,1],c='violet',ls='--')
plt.xlim([-0.05,1.05])
plt.ylim([-0.05,1.05])
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve for ' + type(stacked).__name__);
print("ROC AUC score = {:6.4f}".format(roc_auc_score(y_test, stacked.predict_proba(X_test)[:,1])))
plt.show()

# 7.1 Best model threshold tweaking
### Best model selected based on previous scores

#### 7.1.0 Best model is Gradient Boosting classifier
with Poly transformed dataset and features with at least 5 votes based on feature_selection

best_model=classification(exp_Xpoly_sel_5, y, {0: 1, 1: 15})

##### 7.1.1: Run threshold checking

In [ ]:
X_val, y_val = X_test, y_test # explicitly calling this validation since we're using it for selection

thresh_ps = np.linspace(.10,.50,1000)
model_val_probs = best_model.predict_proba(X_val)[:,1] # positive class probs, same basic logistic model we fit in section 2 

f1_scores, prec_scores, rec_scores, acc_scores = [], [], [], []
for p in thresh_ps:
    model_val_labels = model_val_probs >= p
    f1_scores.append(f1_score(y_val, model_val_labels))    
    prec_scores.append(precision_score(y_val, model_val_labels))
    rec_scores.append(recall_score(y_val, model_val_labels))
    acc_scores.append(accuracy_score(y_val, model_val_labels))
    
plt.plot(thresh_ps, f1_scores)
plt.plot(thresh_ps, prec_scores)
plt.plot(thresh_ps, rec_scores)
plt.plot(thresh_ps, acc_scores)

plt.title('Metric Scores vs. Positive Class Decision Probability Threshold')
plt.legend(['F1','Precision','Recall','Accuracy'])
plt.xlabel('P threshold')
plt.ylabel('Metric score')

best_f1_score = np.max(f1_scores) 
best_thresh_p = thresh_ps[np.argmax(f1_scores)]

print('best_model best F1 score %.3f at prob decision threshold >= %.3f' 
      % (best_f1_score, best_thresh_p))

# 8.1 Deployment and Visualization preparation
### Pickling for web deployment + Writing back to CSV for Tableau viz

##### 8.1.1: Pickle the best model

In [ ]:
bestmodel = best_model

In [ ]:
with open("models/best_classification_model.pickle", "wb") as pfile:
    pickle.dump(bestmodel, pfile)

##### 8.1.2: Adding predicted probability as a column to the dataframe and export as csv

In [ ]:
X_export = exp_Xpoly_sel_5

In [ ]:
X_export.describe()

In [ ]:
X_export['pred_proba'] = bestmodel.predict_proba(X_export)[:, 1]

In [ ]:
X_export.head(5)

In [ ]:
X_export.to_csv("csv_for_tableau/best_classification_model.csv")

# 9.1 Final Result
### visit: https://steam-discount-predictor.herokuapp.com/
### see the Medium blog post: https://medium.com/@opophehu/supervised-classification-a0043d0c5ba5